## Important information
<font color="red"> <b> ⚠ This documment is written as a Jupyter notebook and the code used to produce these analyses may be hidden for ease of readability. The notebook's interactivity as an exported html file is limited. For full widgets-interactivity it needs to run with a python kernel (e.g. jupyter notebook, jupyterlab apps) <br>
    To make the code visible click here:
 </b> </font>

In [ ]:
%%HTML
<script>
code_show=true; 
function code_toggle() {
if (code_show){
$('div.input').hide();
} else {
$('div.input').show();
}
code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Hide/show code"></form>

# Run data preprocessing

In [ ]:
%%capture
%run 03_Analysis_in_the_order_domain.ipynb

# Confidence intervals (on encoder difference)

In this section, the goal is to determine the confidence intervals of the calculated natural frequency by using an Gaussian curve-fitting on the projected data points from the spectrogram analysis on the Frequency-Amplitude plane.

## Method 1: Distribution of the maxima
The simplest method to evaluate the natural frequency value is by computing the maximum for each experiment series separately and conduct a statistical study on the obtained values. Because the number of samples is small and the standard deviation is unknown the parameters for the confidence interval are calculated using the probability density function of the [Student t distribution](https://en.wikipedia.org/wiki/Student%27s_t-distribution 'Wiki page'):

$$ t_{(x,\nu)} = \dfrac{\Gamma \left( \frac{\nu+1}{2} +1\right)}{\sqrt{\nu \pi}\Gamma \left( \frac{\nu}{2}\right)} \left( 1 + \frac{x^2}{\nu} \right)^{-\frac{\nu+1}{2}} $$

With a confidence value of 95%, a number of samples $n$ and a standard deviation $\sigma$ the frequency $f$:

$$f_n = \overline{f_{rpm}}_n \pm t_{(0.05,n-1)} \dfrac{S_n}{\sqrt{\sigma}}  $$

In [ ]:
def distribFit1(Expe, isfiltered=True):
    cut_time = Expe.cut_time
    xdata = []
    ydata = []
    indexdata = []
    fstart = Expe.f*0.75
    fend = Expe.f*1.25
    Nstart = int(fstart*cut_time)
    Nend = int(fend*cut_time)
    N = Nend-Nstart
    maxF = []
    for i in range(Expe.numoffiles):
        if isfiltered:
            indexofmax = Nstart + np.argmax(Expe[i].fft_amp_filt_1minus2[Nstart:Nend])
            maxF.append(Expe[i].fft_freq_filt_1minus2[indexofmax])
        else:
            indexofmax = Nstart + np.argmax(Expe[i].fft_amp_1minus2[Nstart:Nend])
            maxF.append(Expe[i].fft_freq_1minus2[indexofmax])
            
    nof = Expe.numoffiles
    mu = np.mean(maxF)
    sigma = np.std(maxF)
    alpha = t.ppf(0.95, nof-1) * sigma/(np.sqrt(nof))
    
    fig = make_subplots(rows=2, cols=1, row_heights=[0.4,0.8], vertical_spacing=0.01,
                        specs=[[{"type": "table"}],[{"type":"scatter"}]])
    
    table1 = {'Expectation µ = f [Hz]': f"{mu:.2f}" ,
              'Standard deviation σ [Hz]': f"{sigma:.2f}",
              'Confidence α [%]' : '5',
              'Confidence interval [Hz]' : f'{mu:.2f} ± {alpha:.2f}'                           
             }
    df1 = pd.DataFrame(table1, index = ['Value'])
    
    fig.add_trace(go.Scatter(
        showlegend=False,
        x = Expe.speed_list,
        y = maxF,
        text = Expe.f_multiples.round(1),
        textposition="top center",
        mode = 'markers+text',
        marker_color = Expe.f_multiples,
        marker_colorscale = 'bluered',
        marker_colorbar = dict(thickness=20, title='closeness 𝛿f')
        ),2,1
    )
    fig.add_trace(go.Table(header=dict(values=list(table1.keys())),
                 cells=dict(values=list(table1.values()))),
                 1,1)
    fig.update_layout(height = 350, title = f"Frequencies corresponding to the maximum local amplitudes on {'' if isfiltered else 'un'}filtered data<br>" + Expe.shaft_desc, 
                  xaxis1=dict(title='Driven angular velocity'),
                  yaxis1=dict(title='Frequency [Hz]')
                  )
    fig.show()
    display(df1)
    #save_plot_button(fig)

### Healthy shaft

In [ ]:
distribFit1(HeS, isfiltered=False)

In [ ]:
distribFit1(HeS, isfiltered=True)

### Cracked shaft_multiples

In [ ]:
distribFit1(CrS, isfiltered=False)

In [ ]:
distribFit1(CrS, isfiltered=True)

## Method 2: Curve fitting with fixed amplitude on filtered FFT
This method is based on the [curve_fit](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html) function of the [scipy](https://www.scipy.org/) module. The assumption is that the peaks in the frequency spectrograms can be assimilated to a gaussian distribution:

$$A(f,\mu,\sigma, C) =  C\cdot e^{-\dfrac{(f-\mu)^2}{2\sigma^2}}$$

The terms to be determined in the curve fitting are the amplitude $C$, the expectation term $\mu$ and the standard deviation $\sigma$
In this case the datasets for all different driven angular velocities are combined and the parameters are calculated for the joined dataset.

In [ ]:
def distribFit2(Expe):
    cut_time = Expe.cut_time
    xdata = []
    ydata = []
    fstart = 0.5
    fend = Expe.f*2
    Nstart = int(fstart*cut_time)
    Nend = int(fend*cut_time)

    for i in range(Expe.numoffiles):
        xdata.append(Expe[i].fft_freq_filt_1minus2[Nstart:Nend])
        ydata.append(Expe[i].fft_amp_filt_1minus2[Nstart:Nend])
    xdata = np.array(xdata).flatten()
    ydata = np.array(ydata).flatten()
    
    def func(f, C, mu, sigma):
        return C * np.exp(-(f-mu)**2/(2*sigma**2))
    bounds = ([0., Expe.f-5, 1.],[0.02, Expe.f+5, 2.])
    #Fit for the parameters C, mu, sigma of the function func
    popt, pcov = curve_fit(func, xdata, ydata, bounds=bounds)
    
    C, mu, sigma = popt
    
    trace1 = go.Scatter(
        x = xdata,
        y = ydata,
        mode = 'markers',
        marker = dict(size = 3),
        name = 'data points'
    )
    #xdata = xdata[0:Nend]
    trace2 = go.Scatter(
        x = xdata[Nstart+1:Nend-1],
        y = func(xdata, *popt)[Nstart+1:Nend-1],
        mode = 'lines',
        marker = dict(line= dict(width=1), color  = 'orange'),
        name = 'curve fit'
    )
    maxF = 3*np.max(func(xdata, *popt))
    text = f"Expectation µ = f = {mu:.2f}Hz<br>" + \
       f"Standard deviation σ = {sigma:.2f}Hz<br>" + \
       f"Amplitude C = {C:.2f}"
    layout = dict(title = 'Gaussian curve fitting on filtered FFT (single peak single amplitude) -grouped-<br>' + Expe.shaft_desc, 
                  xaxis=dict(title='Freq [Hz]'),
                  yaxis=dict(title='|Y(freq)|', range = [0,maxF]),
                  height=400,
                  annotations = [
                      dict(x=1, y=1, ax=0, ay=0, xref='paper', yref='paper', 
                            text=text, align='right', bgcolor="#ff7f0e",
                            showarrow = False)
                  ]
                  )

    data = [trace1, trace2]
    fig = go.Figure(data=data, layout=layout)
    fig.show()
    save_plot_button(fig)
    

### Healthy shaft

In [ ]:
distribFit2(HeS)

### Cracked shaft

In [ ]:
distribFit2(CrS)

## Method 3: Least square optimization with adapted amplitudes on filtered FFT
For this method, we also try to fit a function to a model like in the previous section, but find the parameters with a least square optimization with the function [least_squares](https://docs.scipy.org/doc/scipy/reference/tutorial/optimize.html) of the [scipy](https://www.scipy.org/) module. Since the amplitudes of the FFT around the frequency zone to be fitted is variable from one dataset to another, the amplitudes are parametrized individually.

### Single peak
- model:

$$A(f,\mu,\sigma, C_{rpm}) =  C_{rpm} \cdot e^{-\dfrac{(f-\mu)^2}{2\sigma^2}}$$


- function to minimize:

$$ \sum{\left( FFT - A(f,\mu,\sigma, C_{rpm}) \right)^2 } $$

The terms to be determined in the optimization are the amplitudes $C_{rpm}$ with $rpm$ ranging from {{HeS.speed_list[0] + ' to ' + HeS.speed_list[-1]}}, the expectation term $\mu$ and the standard deviation $\sigma$

In [ ]:
def distribFit3_sliders(Expe):
    nof = Expe.numoffiles
    cut_time = Expe.cut_time
    xdata = []
    ydata = []
    indexdata = []
    fstart = 0.5
    fend = Expe.f*2
    Nstart = int(fstart*cut_time)
    Nend = int(fend*cut_time)
    N = Nend-Nstart
    for i in range(Expe.numoffiles):
        xdata.append(Expe[i].fft_freq_filt_1minus2[Nstart:Nend])
        ydata.append(Expe[i].fft_amp_filt_1minus2[Nstart:Nend])
        indexdata.append(np.ones((Nend-Nstart,), dtype=np.int)*i)
    
    x = np.array(xdata).flatten()
    y = np.array(ydata).flatten()
    k = np.array(indexdata).flatten()
    u = np.column_stack((k,x))
    
    # define model function
    def model(x, u):
        k = u[:,0].astype(int)
        mu = x[0]
        sigma = x[1]
        C = x[k+2]
        return C * np.exp(-(u[:,1]-mu)**2/(2*sigma**2))
    
    # define function to optimize
    def fun(x, u, y):
        return model(x, u) - y
    # define guesses for the simulation
    x0 = [Expe.f, 2] + [0.02 for i in range(Expe.numoffiles)]
    
    # solve the least square optimization
    res = least_squares(fun,x0, args=(u, y), verbose=1)
    
    # show results
    mu = res.x[0]
    sigma = res.x[1]
    C = res.x[2:]
    
    pd.set_option('display.max_columns', 10)
    table1 = {'Expectation $\mu = f $ [Hz]': mu , 'Standard deviation $\sigma$ [Hz]': sigma}
    table2 = {'Amplitudes $C_{rpm}$' : C}
    df1 = pd.DataFrame(table1, index = ['Value']).round(2)
    df2 = pd.DataFrame(table2,index = HeS.speed_list).T.round(4);
    display(df1,df2)
    
    # plot the data
    trace0 = [] ; trace1 = []
    for i in range(Expe.numoffiles):
        E=Expe[i]
        trace0.append(go.Scatter(
            visible = False,
            x = x[i*N:(i+1)*N],
            y = y[i*N:(i+1)*N],
            mode = 'markers',
            marker =  dict(size= 3),
            name = E.speed_rpm_str + ' data points '
        ))
        trace1.append(go.Scatter(
            visible = False,
            x = x[i*N:(i+1)*N],
            y = model(res.x, u)[i*N:(i+1)*N],
            mode = 'lines',
            marker= dict(line= dict(width=1)),
            name =  E.speed_rpm_str + ' curve fit'
            ))

    data = trace0 + trace1

    data[0]['visible'] = True
    data[nof]['visible'] = True

    steps = []
    for i in range(nof):
        step = dict(
            method = 'restyle',
            args = ['visible', [False] * len(data)],
            label=Expe[i].speed_rpm_str
        )
        step['args'][1][i] = True # Toggle i'th trace to "visible"
        step['args'][1][i+nof] = True # Toggle i'th trace to "visible"
        steps.append(step)
    
    sliders = [dict(
        active = 0,
        currentvalue = dict(prefix = "Driven angular velocity: "),
        steps = steps,
        pad = {"t": 80}
    )]
    
    updatemenus = [
        dict(buttons= [
            dict(
                args = ['visible', [True] * len(data)],
                label='See all traces',
                method='restyle'
                )],
            direction = 'left',
            pad = {'r': 10, 't': 10},
            showactive = False,
            type = 'buttons',
            x = 0.1,
            xanchor = 'left',
            y = 1,
            yanchor = 'top' 
        )
    ]
    maxF = np.max(model(res.x, u))
    layout = dict(hovermode = 'closest',
                  updatemenus = updatemenus,
                  sliders = sliders, 
                  title = 'Gaussian curve-fitting on the projected data points from the spectrogram analysis on the Frequency-Amplitude plane <br>' + Expe.shaft_desc, 
                  xaxis=dict(title='Freq [Hz]'),
                  yaxis=dict(title='|Y(freq)|', range = [0,1.5*maxF])
                  )

    fig = go.Figure(data=data, layout=layout)
    fig.show()
    
    save_plot_button(fig)
    

In [ ]:
def distribFit3_single(Expe):
    nof = Expe.numoffiles
    cut_time = Expe.cut_time
    xdata = []
    ydata = []
    indexdata = []
    fstart = 0.5
    fend = Expe.f*2
    Nstart = int(fstart*cut_time)
    Nend = int(fend*cut_time)
    N = Nend-Nstart
    for i in range(Expe.numoffiles):
        xdata.append(Expe[i].fft_freq_filt_1minus2[Nstart:Nend])
        ydata.append(Expe[i].fft_amp_filt_1minus2[Nstart:Nend])
        indexdata.append(np.ones((Nend-Nstart,), dtype=np.int)*i)
    
    x = np.array(xdata).flatten()
    y = np.array(ydata).flatten()
    k = np.array(indexdata).flatten()
    u = np.column_stack((k,x))
    
    # define model function
    def model(x, u):
        k = u[:,0].astype(int)
        mu = x[0]
        sigma = x[1]
        C = x[k+2]
        return C * np.exp(-(u[:,1]-mu)**2/(2*sigma**2))
    
    # define function to optimize
    def fun(x, u, y):
        return model(x, u) - y
    # define guesses for the simulation
    x0 = [Expe.f, 2] + [0.02 for i in range(Expe.numoffiles)]
    
    # solve the least square optimization
    res = least_squares(fun,x0, args=(u, y), verbose=1)
    
    # show results
    mu = res.x[0]
    sigma = res.x[1]
    C = res.x[2:]
    
    # plot the data
    trace0 = [] ; trace1 = []
    for i in range(Expe.numoffiles):
        E=Expe[i]
        trace0.append(go.Scatter(
            x = x[i*N:(i+1)*N],
            y = y[i*N:(i+1)*N],
            mode = 'markers',
            marker =  dict(size= 3),
            name = E.speed_rpm_str + ' data points '
        ))
        trace1.append(go.Scatter(
            x = x[i*N:(i+1)*N],
            y = model(res.x, u)[i*N:(i+1)*N],
            mode = 'lines',
            marker= dict(line= dict(width=1)),
            name =  E.speed_rpm_str + ' curve fit'
            ))

    data = trace0 + trace1

    maxF = np.max(model(res.x, u))
    text = f"Expectation µ = f = {mu:.2f}Hz<br>" + \
       f"Standard deviation σ = {sigma:.2f}Hz"
    layout = dict(title = 'Least square optimization on filtered FFT (single peak) -grouped-<br>' + Expe.shaft_desc, 
                  xaxis=dict(title='Freq [Hz]'),
                  yaxis=dict(title='|Y(freq)|', range = [0,maxF]),
                  height=400,
                  annotations = [
                      dict(x=1, y=1, ax=0, ay=0, xref='paper', yref='paper', 
                            text=text, align='right', bgcolor="#ff7f0e",
                            showarrow = False)
                  ]
                 )

    fig = go.Figure(data=data, layout=layout)
    fig.show()
    
    save_plot_button(fig)

In [ ]:
def distribFit3(Expe, yrange=None, log = False):
    fig = make_subplots(rows=5, cols=6, subplot_titles=(Expe.speed_list),
                        horizontal_spacing = 0.01, vertical_spacing=0.05,
                        shared_yaxes=True, x_title='Freq [Hz]', y_title='|Y(freq)|')
    
    nof = Expe.numoffiles
    cut_time = Expe.cut_time
    xdata = []
    ydata = []
    indexdata = []
    fstart = 0.5
    fend = Expe.f*2
    Nstart = int(fstart*cut_time)
    Nend = int(fend*cut_time)
    N = Nend-Nstart
    for i in range(Expe.numoffiles):
        xdata.append(Expe[i].fft_freq_filt_1minus2[Nstart:Nend])
        ydata.append(Expe[i].fft_amp_filt_1minus2[Nstart:Nend])
        indexdata.append(np.ones((Nend-Nstart,), dtype=np.int)*i)
    
    x = np.array(xdata).flatten()
    y = np.array(ydata).flatten()
    k = np.array(indexdata).flatten()
    u = np.column_stack((k,x))
    
    # define model function
    def model(x, u):
        k = u[:,0].astype(int)
        mu = x[0]
        sigma = x[1]
        C = x[k+2]
        return C * np.exp(-(u[:,1]-mu)**2/(2*sigma**2))
    
    # define function to optimize
    def fun(x, u, y):
        return model(x, u) - y
    # define guesses for the simulation
    x0 = [Expe.f, 2] + [0.02 for i in range(Expe.numoffiles)]
    
    # solve the least square optimization
    res = least_squares(fun,x0, args=(u, y), verbose=1)
    
    # show results
    mu = res.x[0]
    sigma = res.x[1]
    C = res.x[2:]
    
    k=0
    for i in range(1,6):
        for j in range(1,7):
            E = Expe[k]
            NF = int(E.t1a[-1]*60)
            fig.add_trace(go.Scatter(x = x[k*N:(k+1)*N],
                                     y = y[k*N:(k+1)*N],
                                     mode = 'markers',
                                     marker =  dict(size= 1, color = 'blue'),
                                     name = 'data points',
                                     legendgroup="data",
                                     showlegend = True if k==0 else False
                                    ),
                          row=i, col=j
                         )
            fig.add_trace(go.Scatter(x = x[k*N:(k+1)*N],
                                     y = model(res.x, u)[k*N:(k+1)*N],
                                     mode = 'lines',
                                     name =  'curve fit',
                                     line = dict( color = 'red', width = 1),
                                     legendgroup="fit",
                                     showlegend = True if k==0 else False
                                    ),
                          row=i, col=j
                         )
            
            
            k+=1

    for a in fig.layout.annotations:
        a.font.size = 12
    if log == False and yrange!=None:
        for i in range(1,31):
            fig.layout[f'yaxis{i}'].range = (0,yrange)
    fig.update_layout(title_text = 'Least square optimization on filtered FFT (single peak) -matrix-<br>' + Expe.shaft_desc,
                      height=800, width=1000, font_size=10, legend_orientation="h")
    fig.show()
    
    save_plot_button(fig)

#### Healthy shaft

In [ ]:
distribFit3_single(HeS)

In [ ]:
distribFit3(HeS, yrange=0.04)

#### Cracked shaft

In [ ]:
distribFit3_single(CrS)

In [ ]:
distribFit3(CrS)

### Single peak, single wedge slope
In the FFT spectrum we can see that the noisy data at low amplitudes tend to increase linearly at higher frequencies. Therefore we can adapt our model and add this further parameter to it. In this case a single slope value is parametrized for all the datasets.

- model:

$$A(f,\mu,\sigma, C_{rpm}) =  C_{rpm} \cdot e^{-\dfrac{(f-\mu)^2}{2\sigma^2}} + c\cdot f $$


- function to minimize:

$$ \sum{\left( FFT - A(f,\mu,\sigma, C_{rpm},c) \right)^2 } $$

The terms to be determined in the optimization are the amplitudes $C_{rpm}$ with $rpm$ ranging from {{HeS.speed_list[0] + ' to ' + HeS.speed_list[-1]}}, the expectation term $\mu$, the standard deviation $\sigma$ and the wedge slope $c$

In [ ]:
def distribFit31_sliders(Expe):
    nof = Expe.numoffiles
    cut_time = Expe.cut_time
    xdata = []
    ydata = []
    indexdata = []
    fstart = 0.5
    fend = Expe.f*2
    Nstart = int(fstart*cut_time)
    Nend = int(fend*cut_time)
    N = Nend-Nstart
    for i in range(Expe.numoffiles):
        xdata.append(Expe[i].fft_freq_filt_1minus2[Nstart:Nend])
        ydata.append(Expe[i].fft_amp_filt_1minus2[Nstart:Nend])
        indexdata.append(np.ones((Nend-Nstart,), dtype=np.int)*i)
    
    x = np.array(xdata).flatten()
    y = np.array(ydata).flatten()
    k = np.array(indexdata).flatten()
    u = np.column_stack((k,x))
    
    # define model function
    def model(x, u):
        k = u[:,0].astype(int)
        mu = x[0]
        sigma = x[1]
        c = x[2]
        C = x[k+3]
        return C * np.exp(-(u[:,1]-mu)**2/(2*sigma**2)) + c*u[:,1]
    
    # define function to optimize
    def fun(x, u, y):
        return model(x, u) - y
    
    # define guesses for the simulation
    x0 = [Expe.f, 2, 0.0001] + [0.02 for i in range(nof)]
    
    # solve the least square optimization
    res = least_squares(fun,x0, args=(u, y), verbose=1)
    
    # show results
    mu = res.x[0]
    sigma = res.x[1]
    c = res.x[2]
    C = res.x[3:]
    
    pd.set_option('display.max_columns', 10)
    table1 = {'Expectation $\mu = f $ [Hz]': mu , 
              'Standard deviation $\sigma$ [Hz]': sigma,
              'Wedge slope $c \cdot 10^3$': c*1000}
    table2 = {'Amplitudes $C_{rpm}$' : C}
    df1 = pd.DataFrame(table1, index = ['Value']).round(2)
    df2 = pd.DataFrame(table2,index = HeS.speed_list).T.round(4);
    display(df1,df2)
    
    # plot the data
    trace0 = [] ; trace1 = []
    for i in range(Expe.numoffiles):
        E=Expe[i]
        trace0.append(go.Scatter(
            visible = False,
            x = x[i*N:(i+1)*N],
            y = y[i*N:(i+1)*N],
            mode = 'markers',
            marker =  dict(size= 3),
            name = E.speed_rpm_str + ' data points '
        ))
        trace1.append(go.Scatter(
            visible = False,
            x = x[i*N:(i+1)*N],
            y = model(res.x, u)[i*N:(i+1)*N],
            mode = 'lines',
            marker= dict(line= dict(width=1)),
            name =  E.speed_rpm_str + ' curve fit'
            ))

    data = trace0 + trace1

    data[0]['visible'] = True
    data[nof]['visible'] = True

    steps = []
    for i in range(nof):
        step = dict(
            method = 'restyle',
            args = ['visible', [False] * len(data)],
            label=Expe[i].speed_rpm_str
        )
        step['args'][1][i] = True # Toggle i'th trace to "visible"
        step['args'][1][i+nof] = True # Toggle i'th trace to "visible"
        steps.append(step)
    
    sliders = [dict(
        active = 0,
        currentvalue = dict(prefix = "Driven angular velocity: "),
        steps = steps,
        pad = {"t": 80}
    )]
    
    updatemenus = [
        dict(buttons= [
            dict(
                args = ['visible', [True] * len(data)],
                label='See all traces',
                method='restyle'
                )],
            direction = 'left',
            pad = {'r': 10, 't': 10},
            showactive = False,
            type = 'buttons',
            x = 0.1,
            xanchor = 'left',
            y = 1,
            yanchor = 'top' 
        )
    ]
    maxF = np.max(model(res.x, u))
    layout = dict(hovermode = 'closest',
                  updatemenus = updatemenus,
                  sliders = sliders, 
                  title = 'Gaussian curve-fitting on the projected data points from the spectrogram analysis on the Frequency-Amplitude plane <br>' + Expe.shaft_desc, 
                  xaxis=dict(title='Freq [Hz]'),
                  yaxis=dict(title='|Y(freq)|', range = [0,1.5*maxF])
                  )

    fig = go.Figure(data=data, layout=layout)
    fig.show()
    
    save_plot_button(fig)
    

In [ ]:
def distribFit31_single(Expe):
    nof = Expe.numoffiles
    cut_time = Expe.cut_time
    xdata = []
    ydata = []
    indexdata = []
    fstart = 0.5
    fend = Expe.f*2
    Nstart = int(fstart*cut_time)
    Nend = int(fend*cut_time)
    N = Nend-Nstart
    for i in range(Expe.numoffiles):
        xdata.append(Expe[i].fft_freq_filt_1minus2[Nstart:Nend])
        ydata.append(Expe[i].fft_amp_filt_1minus2[Nstart:Nend])
        indexdata.append(np.ones((Nend-Nstart,), dtype=np.int)*i)
    
    x = np.array(xdata).flatten()
    y = np.array(ydata).flatten()
    k = np.array(indexdata).flatten()
    u = np.column_stack((k,x))
    
    # define model function
    def model(x, u):
        k = u[:,0].astype(int)
        mu = x[0]
        sigma = x[1]
        c = x[2]
        C = x[k+3]
        return C * np.exp(-(u[:,1]-mu)**2/(2*sigma**2)) + c*u[:,1]
    
    # define function to optimize
    def fun(x, u, y):
        return model(x, u) - y
    
    # define guesses for the simulation
    x0 = [Expe.f, 2, 0.0001] + [0.02 for i in range(nof)]
    
    # solve the least square optimization
    res = least_squares(fun,x0, args=(u, y), verbose=1)
    
    # show results
    mu = res.x[0]
    sigma = res.x[1]
    c = res.x[2]
    C = res.x[3:]
    
    # plot the data
    trace0 = [] ; trace1 = []
    for i in range(Expe.numoffiles):
        E=Expe[i]
        trace0.append(go.Scatter(
            x = x[i*N:(i+1)*N],
            y = y[i*N:(i+1)*N],
            mode = 'markers',
            marker =  dict(size= 3),
            name = E.speed_rpm_str + ' data points '
        ))
        trace1.append(go.Scatter(
            x = x[i*N:(i+1)*N],
            y = model(res.x, u)[i*N:(i+1)*N],
            mode = 'lines',
            marker= dict(line= dict(width=1)),
            name =  E.speed_rpm_str + ' curve fit'
            ))

    data = trace0 + trace1

    maxF = np.max(model(res.x, u))
    text = f"Expectation µ = f = {mu:.2f}Hz<br>" + \
       f"Standard deviation σ = {sigma:.2f}Hz"
    layout = dict(title = 'Least square optimization on filtered FFT (single peak single wedge) -grouped-<br>' + Expe.shaft_desc, 
                  xaxis=dict(title='Freq [Hz]'),
                  yaxis=dict(title='|Y(freq)|', range = [0,maxF]),
                  height=400,
                  annotations = [
                      dict(x=1, y=1, ax=0, ay=0, xref='paper', yref='paper', 
                            text=text, align='right', bgcolor="#ff7f0e",
                            showarrow = False)
                  ]
                 )

    fig = go.Figure(data=data, layout=layout)
    fig.show()
    
    save_plot_button(fig)

In [ ]:
def distribFit31(Expe, yrange=None, log = False):
    fig = make_subplots(rows=5, cols=6, subplot_titles=(Expe.speed_list),
                        horizontal_spacing = 0.01, vertical_spacing=0.05,
                        shared_yaxes=True, x_title='Freq [Hz]', y_title='|Y(freq)|')
    
    nof = Expe.numoffiles
    cut_time = Expe.cut_time
    xdata = []
    ydata = []
    indexdata = []
    fstart = 0.5
    fend = Expe.f*2
    Nstart = int(fstart*cut_time)
    Nend = int(fend*cut_time)
    N = Nend-Nstart
    for i in range(Expe.numoffiles):
        xdata.append(Expe[i].fft_freq_filt_1minus2[Nstart:Nend])
        ydata.append(Expe[i].fft_amp_filt_1minus2[Nstart:Nend])
        indexdata.append(np.ones((Nend-Nstart,), dtype=np.int)*i)
    
    x = np.array(xdata).flatten()
    y = np.array(ydata).flatten()
    k = np.array(indexdata).flatten()
    u = np.column_stack((k,x))
    
    # define model function
    def model(x, u):
        k = u[:,0].astype(int)
        mu = x[0]
        sigma = x[1]
        c = x[2]
        C = x[k+3]
        return C * np.exp(-(u[:,1]-mu)**2/(2*sigma**2)) + c*u[:,1]
    
    # define function to optimize
    def fun(x, u, y):
        return model(x, u) - y
    
    # define guesses for the simulation
    x0 = [Expe.f, 2, 0.0001] + [0.02 for i in range(nof)]
    
    # solve the least square optimization
    res = least_squares(fun,x0, args=(u, y), verbose=1)
    
    # show results
    mu = res.x[0]
    sigma = res.x[1]
    c = res.x[2]
    C = res.x[3:]
    
    k=0
    for i in range(1,6):
        for j in range(1,7):
            E = Expe[k]
            NF = int(E.t1a[-1]*60)
            fig.add_trace(go.Scatter(x = x[k*N:(k+1)*N],
                                     y = y[k*N:(k+1)*N],
                                     mode = 'markers',
                                     marker =  dict(size= 1, color = 'blue'),
                                     name = 'data points',
                                     legendgroup="data",
                                     showlegend = True if k==0 else False
                                    ),
                          row=i, col=j
                         )
            fig.add_trace(go.Scatter(x = x[k*N:(k+1)*N],
                                     y = model(res.x, u)[k*N:(k+1)*N],
                                     mode = 'lines',
                                     name =  'curve fit',
                                     line = dict( color = 'red', width = 1),
                                     legendgroup="fit",
                                     showlegend = True if k==0 else False
                                    ),
                          row=i, col=j
                         )
            
            
            k+=1

    for a in fig.layout.annotations:
        a.font.size = 12
    if log == False and yrange!=None:
        for i in range(1,31):
            fig.layout[f'yaxis{i}'].range = (0,yrange)
    fig.update_layout(title_text = 'Least square optimization on filtered FFT (single peak single wedge)<br>' + Expe.shaft_desc,
                      height=800, width=1000, font_size=10, legend_orientation="h")
    fig.show()
    
    save_plot_button(fig)

#### Healthy shaft

In [ ]:
distribFit31_single(HeS)

In [ ]:
distribFit31(HeS)

#### Cracked shaft

In [ ]:
distribFit31_single(CrS)

In [ ]:
distribFit31(CrS)

### Single peak, multiple wedge slopes
In the FFT spectrum we can see that the noisy data at low amplitudes tend to increase linearly at higher frequencies. Therefore we can adapt our model and add this further parameter to it. In this case a slope value is parametrized for each series of  the datasets.

- model:

$$A(f,\mu,\sigma, C_{rpm}) =  C_{rpm} \cdot e^{-\dfrac{(f-\mu)^2}{2\sigma^2}} + c_{rpm}\cdot f $$


- function to minimize:

$$ \sum{\left( FFT - A(f,\mu,\sigma, C_{rpm},c_{rpm}) \right)^2 } $$

The terms to be determined in the optimization are the amplitudes $C_{rpm}$ and the wedge slopes $c_{rpm}$ with $rpm$ ranging from {{HeS.speed_list[0] + ' to ' + HeS.speed_list[-1]}}, the expectation term $\mu$, the standard deviation $\sigma$.

In [ ]:
def distribFit32_sliders(Expe):
    nof = Expe.numoffiles
    cut_time = Expe.cut_time
    xdata = []
    ydata = []
    indexdata = []
    fstart = 0.5
    fend = Expe.f*2
    Nstart = int(fstart*cut_time)
    Nend = int(fend*cut_time)
    N = Nend-Nstart
    for i in range(Expe.numoffiles):
        xdata.append(Expe[i].fft_freq_filt_1minus2[Nstart:Nend])
        ydata.append(Expe[i].fft_amp_filt_1minus2[Nstart:Nend])
        indexdata.append(np.ones((Nend-Nstart,), dtype=np.int)*i)
    
    x = np.array(xdata).flatten()
    y = np.array(ydata).flatten()
    k = np.array(indexdata).flatten()
    u = np.column_stack((k,x))
    
    # define model function
    def model(x, u):
        k = u[:,0].astype(int)
        mu = x[0]
        sigma = x[1]
        c = x[k+2]
        C = x[k+nof+2]
        return C * np.exp(-(u[:,1]-mu)**2/(2*sigma**2)) + c*u[:,1]
    
    # define function to optimize
    def fun(x, u, y):
        return model(x, u) - y
    
    # define guesses for the simulation
    x0 = [Expe.f, 2] + [0.0001 for i in range(nof)] + [0.02 for i in range(nof)]
    
    # solve the least square optimization
    res = least_squares(fun,x0, args=(u, y), verbose=1)
    
    # show results
    mu = res.x[0]
    sigma = res.x[1]
    c = res.x[2:2+nof]
    C = res.x[2+nof:2+2*nof]
    
    pd.set_option('display.max_columns', 10)
    table1 = {'Expectation $\mu = f $ [Hz]': mu , 
              'Standard deviation $\sigma$ [Hz]': sigma,}
    table2 = {'Amplitudes $C_{rpm}$' : C,
              'Wedge slope $c \cdot 10^3$': c*1000}
    df1 = pd.DataFrame(table1, index = ['Value']).round(2)
    df2 = pd.DataFrame(table2,index = HeS.speed_list).T.round(4);
    display(df1,df2)
    
    # plot the data
    trace0 = [] ; trace1 = []
    for i in range(Expe.numoffiles):
        E=Expe[i]
        trace0.append(go.Scatter(
            visible = False,
            x = x[i*N:(i+1)*N],
            y = y[i*N:(i+1)*N],
            mode = 'markers',
            marker =  dict(size= 3),
            name = E.speed_rpm_str + ' data points '
        ))
        trace1.append(go.Scatter(
            visible = False,
            x = x[i*N:(i+1)*N],
            y = model(res.x, u)[i*N:(i+1)*N],
            mode = 'lines',
            marker= dict(line= dict(width=1)),
            name =  E.speed_rpm_str + ' curve fit'
            ))

    data = trace0 + trace1

    data[0]['visible'] = True
    data[nof]['visible'] = True

    steps = []
    for i in range(nof):
        step = dict(
            method = 'restyle',
            args = ['visible', [False] * len(data)],
            label=Expe[i].speed_rpm_str
        )
        step['args'][1][i] = True # Toggle i'th trace to "visible"
        step['args'][1][i+nof] = True # Toggle i'th trace to "visible"
        steps.append(step)
    
    sliders = [dict(
        active = 0,
        currentvalue = dict(prefix = "Driven angular velocity: "),
        steps = steps,
        pad = {"t": 80}
    )]
    
    updatemenus = [
        dict(buttons= [
            dict(
                args = ['visible', [True] * len(data)],
                label='See all traces',
                method='restyle'
                )],
            direction = 'left',
            pad = {'r': 10, 't': 10},
            showactive = False,
            type = 'buttons',
            x = 0.1,
            xanchor = 'left',
            y = 1,
            yanchor = 'top' 
        )
    ]
    maxF = np.max(model(res.x, u))
    layout = dict(hovermode = 'closest',
                  updatemenus = updatemenus,
                  sliders = sliders, 
                  title = 'Gaussian curve-fitting on the projected data points from the spectrogram analysis on the Frequency-Amplitude plane <br>' + Expe.shaft_desc, 
                  xaxis=dict(title='Freq [Hz]'),
                  yaxis=dict(title='|Y(freq)|', range = [0,1.5*maxF])
                  )

    fig = go.Figure(data=data, layout=layout)
    fig.show()
    
    save_plot_button(fig)
    

In [ ]:
def distribFit32_single(Expe):
    nof = Expe.numoffiles
    cut_time = Expe.cut_time
    xdata = []
    ydata = []
    indexdata = []
    fstart = 0.5
    fend = Expe.f*2
    Nstart = int(fstart*cut_time)
    Nend = int(fend*cut_time)
    N = Nend-Nstart
    for i in range(Expe.numoffiles):
        xdata.append(Expe[i].fft_freq_filt_1minus2[Nstart:Nend])
        ydata.append(Expe[i].fft_amp_filt_1minus2[Nstart:Nend])
        indexdata.append(np.ones((Nend-Nstart,), dtype=np.int)*i)
    
    x = np.array(xdata).flatten()
    y = np.array(ydata).flatten()
    k = np.array(indexdata).flatten()
    u = np.column_stack((k,x))
    
    # define model function
    def model(x, u):
        k = u[:,0].astype(int)
        mu = x[0]
        sigma = x[1]
        c = x[k+2]
        C = x[k+nof+2]
        return C * np.exp(-(u[:,1]-mu)**2/(2*sigma**2)) + c*u[:,1]
    
    # define function to optimize
    def fun(x, u, y):
        return model(x, u) - y
    
    # define guesses for the simulation
    x0 = [Expe.f, 2] + [0.0001 for i in range(nof)] + [0.02 for i in range(nof)]
    
    # solve the least square optimization
    res = least_squares(fun,x0, args=(u, y), verbose=1)
    
    # show results
    mu = res.x[0]
    sigma = res.x[1]
    c = res.x[2:2+nof]
    C = res.x[2+nof:2+2*nof]
    
    # plot the data
    trace0 = [] ; trace1 = []
    for i in range(Expe.numoffiles):
        E=Expe[i]
        trace0.append(go.Scatter(
            x = x[i*N:(i+1)*N],
            y = y[i*N:(i+1)*N],
            mode = 'markers',
            marker =  dict(size= 3),
            name = E.speed_rpm_str + ' data points '
        ))
        trace1.append(go.Scatter(
            x = x[i*N:(i+1)*N],
            y = model(res.x, u)[i*N:(i+1)*N],
            mode = 'lines',
            marker= dict(line= dict(width=1)),
            name =  E.speed_rpm_str + ' curve fit'
            ))

    data = trace0 + trace1

    maxF = np.max(model(res.x, u))
    text = f"Expectation µ = f = {mu:.2f}Hz<br>" + \
       f"Standard deviation σ = {sigma:.2f}Hz"
    layout = dict(title = 'Least square optimization on filtered FFT (single peak multiple wedges) -grouped-<br>' + Expe.shaft_desc, 
                  xaxis=dict(title='Freq [Hz]'),
                  yaxis=dict(title='|Y(freq)|', range = [0,maxF]),
                  height=400,
                  annotations = [
                      dict(x=1, y=1, ax=0, ay=0, xref='paper', yref='paper', 
                            text=text, align='right', bgcolor="#ff7f0e",
                            showarrow = False)
                  ]
                 )

    fig = go.Figure(data=data, layout=layout)
    fig.show()
    
    save_plot_button(fig)

In [ ]:
def distribFit32(Expe, yrange=None, log = False):
    fig = make_subplots(rows=5, cols=6, subplot_titles=(Expe.speed_list),
                        horizontal_spacing = 0.01, vertical_spacing=0.05,
                        shared_yaxes=True, x_title='Freq [Hz]', y_title='|Y(freq)|')
    
    nof = Expe.numoffiles
    cut_time = Expe.cut_time
    xdata = []
    ydata = []
    indexdata = []
    fstart = 0.5
    fend = Expe.f*2
    Nstart = int(fstart*cut_time)
    Nend = int(fend*cut_time)
    N = Nend-Nstart
    for i in range(Expe.numoffiles):
        xdata.append(Expe[i].fft_freq_filt_1minus2[Nstart:Nend])
        ydata.append(Expe[i].fft_amp_filt_1minus2[Nstart:Nend])
        indexdata.append(np.ones((Nend-Nstart,), dtype=np.int)*i)
    
    x = np.array(xdata).flatten()
    y = np.array(ydata).flatten()
    k = np.array(indexdata).flatten()
    u = np.column_stack((k,x))
    
    # define model function
    def model(x, u):
        k = u[:,0].astype(int)
        mu = x[0]
        sigma = x[1]
        c = x[k+2]
        C = x[k+nof+2]
        return C * np.exp(-(u[:,1]-mu)**2/(2*sigma**2)) + c*u[:,1]
    
    # define function to optimize
    def fun(x, u, y):
        return model(x, u) - y
    
    # define guesses for the simulation
    x0 = [Expe.f, 2] + [0.0001 for i in range(nof)] + [0.02 for i in range(nof)]
    
    # solve the least square optimization
    res = least_squares(fun,x0, args=(u, y), verbose=1)
    
    # show results
    mu = res.x[0]
    sigma = res.x[1]
    c = res.x[2:2+nof]
    C = res.x[2+nof:2+2*nof]
    
    k=0
    for i in range(1,6):
        for j in range(1,7):
            E = Expe[k]
            NF = int(E.t1a[-1]*60)
            fig.add_trace(go.Scatter(x = x[k*N:(k+1)*N],
                                     y = y[k*N:(k+1)*N],
                                     mode = 'markers',
                                     marker =  dict(size= 1, color = 'blue'),
                                     name = 'data points',
                                     legendgroup="data",
                                     showlegend = True if k==0 else False
                                    ),
                          row=i, col=j
                         )
            fig.add_trace(go.Scatter(x = x[k*N:(k+1)*N],
                                     y = model(res.x, u)[k*N:(k+1)*N],
                                     mode = 'lines',
                                     name =  'curve fit',
                                     line = dict( color = 'red', width = 1),
                                     legendgroup="fit",
                                     showlegend = True if k==0 else False
                                    ),
                          row=i, col=j
                         )
            
            
            k+=1

    for a in fig.layout.annotations:
        a.font.size = 12
    if log == False and yrange!=None:
        for i in range(1,31):
            fig.layout[f'yaxis{i}'].range = (0,yrange)
    fig.update_layout(title_text = 'Least square optimization on filtered FFT (single peak multiple wedges)<br>' + Expe.shaft_desc,
                      height=800, width=1000, font_size=10, legend_orientation="h")
    fig.show()
    
    save_plot_button(fig)

#### Healthy shaft

In [ ]:
distribFit32_single(HeS)

In [ ]:
distribFit32(HeS)

#### Cracked shaft

In [ ]:
distribFit32_single(CrS)

In [ ]:
distribFit32(CrS)

## Method 4: Least square optimization with adapted amplitudes on unfiltered FFT
Similarly to the previous method, we also try to fit a function to a model and find the parameters with a least square optimization with the function [least_squares](https://docs.scipy.org/doc/scipy/reference/tutorial/optimize.html) of the [scipy](https://www.scipy.org/) module. This time the optimization is intended to be on the unfiltered signal where the orders are present and as seen in the 3D-spectrograms represent most of the signal. Since the amplitudes $C1(rpm)$ of the FFT around the frequency zone to be fitted and the amplitudes $C2(rpm)$ of the orders are variable from one dataset to another, they are parametrized individually. The parameters $\mu_1$, $\sigma1$, and $\sigma2$ on the other hand are set to be in common for all the sets. 

Additionally the frequency domain has be carefully chosen so that only the first order for each experiment series appears per dataset, along with the peak of the natural frequency to be found.

### Multiple peaks
- Model:
$$A(f, rpm, \mu_1,\sigma_1, \sigma_2, C_{1_{rpm}}, C_{2_{rpm}}) =  C_{1_{rpm}} \cdot e^{-\dfrac{(f-\mu_1)^2}{2\sigma_1^2}} + C_{2_{rpm}} \cdot e^{-\dfrac{(f-k\cdot \frac{rpm}{60})^2}{2\sigma_2^2}}$$ 

where $k$ represent the order to be fitted and $rpm$ the driven angular velocity in rounds per minutes


- Function to minimize:

$$ \sum_{rpm,f}{ \left( FFT(f, rpm) - A(f, rpm, \mu_1,\sigma_1, \sigma_2, C_{1_{rpm}}, C_{2_{rpm}}) \right)^2 }  $$

The terms to be determined in the optimization are the amplitudes $C_{rpm}$ with $rpm$ ranging from {{HeS.speed_list[0] + ' to ' + HeS.speed_list[-1]}}, the expectation term $\mu$ and the standard deviation $\sigma$

In [ ]:
def distribFit4_sliders(Expe):
    nof = Expe.numoffiles
    cut_time = Expe.cut_time
    xdata = []
    ydata = []
    indexdata = []
    fstart = 0.5
    fend = Expe.f*1.5
    Nstart = int(fstart*cut_time)
    Nend = int(fend*cut_time)
    N = Nend-Nstart
    for i in range(nof):
        xdata.append(Expe[i].fft_freq_1minus2_cut[Nstart:Nend])
        ydata.append(Expe[i].fft_amp_1minus2_cut[Nstart:Nend])
        indexdata.append(np.ones((Nend-Nstart,), dtype=np.int)*(i))

    x = np.array(xdata).flatten()
    y = np.array(ydata).flatten()
    k = np.array(indexdata).flatten()
    u = np.column_stack((k,x))
    k = u[:,0].astype(int)
    X = u[:,1]
    mu2v = np.array([Expe[i].speed_rpm_num/60 for i in range(nof)])
    # define model function
    def model(x):
        mu1 = x[0]
        sigma1 = x[1]
        sigma2 = x[2]
        C1 = x[k+3]
        C2 = x[k+nof+3]
        mu2 = mu2v[k]
        return np.abs(C1) * np.exp(-(X-mu1)**2/(2*sigma1**2)) + np.abs(C2) * np.exp(-(X-mu2)**2/(2*sigma2**2))

    # define function to optimize
    def func(x):
        return model(x) - y
               
    def funcscal(x):
        return np.sum(np.power((model(x) - y), 2))
    
    
    # define guesses for the simulation
    mu1 = [Expe.f]
    sigma1 = [5]
    sigma2 = [1]
    C1 = [0.05 for i in range(nof)]
    C2 = [Expe[i].speed_rpm_num*0.0002 for i in range(nof)]

    x0 = mu1 + sigma1 + sigma2 + C1 + C2
    a= 0.2
    b= 5
    a1= 0.8
    b1= 1.2
    bmu1 = [(mu1[0]*0.9, mu1[0]*1.1)]
    bsigma1 = [(0, sigma1[0]*b1)]
    bsigma2 = [(0, sigma2[0]*b1)]
    bC1 = [(0, C1[i]*b) for i in range(nof)]
    bC2 = [(0, C2[i]*b) for i in range(nof)]

    bounds = bmu1 + bsigma1 + bsigma2  + bC1 + bC2
    
    # solve the least square optimization
    #res = least_squares(func,x0)
    res = minimize(funcscal,x0, method='Nelder-Mead',tol=1e-6)
    # res = differential_evolution(funcscal, bounds)
    
    # show results
    mu1 = res.x[0]
    sigma1 = res.x[1]
    sigma2 = res.x[2]
    C1 = res.x[3:nof+3]
    C2 = res.x[nof+3:2*nof+3]
    
    # plot the results in a table
    pd.set_option('display.max_columns', 30)
    table1 = {'Expectation $\mu_1 = f $ [Hz]': mu1,
              'Standard deviation $\sigma1$ [Hz]': sigma1,
              'Standard deviation $\sigma2$ [Hz]': sigma2}
    table2 = {'Expectation $\mu_2 = f $ [Hz]': mu2v,
              'Amplitudes $C_{1_{rpm}}$' : C1,
              'Amplitudes $C_{2_{rpm}}$' : C2}

    df1 = pd.DataFrame(table1, index = ['Value']).round(2)
    df2 = pd.DataFrame(table2,index = Expe.speed_list).T.round(4);
    display(df1,df2)

    #plot the results
    trace0 = [] ; trace1 = [] ; trace2 = []

    for i in range(Expe.numoffiles):
        E=Expe[i]
        trace0.append(go.Scatter(
            visible = False,
            x = X[i*N:(i+1)*N],
            y = y[i*N:(i+1)*N],
            mode = 'markers',
            marker =  dict(size= 3),
            name = E.speed_rpm_str + ' data points'
        ))
        trace1.append(go.Scatter(
            visible = False,
            x = X[i*N:(i+1)*N],
            y = model(res.x)[i*N:(i+1)*N],
            mode = 'lines',
            marker= dict(line= dict(width=1)),
            name =  E.speed_rpm_str + ' curve fit'
        ))
        trace2.append(go.Scatter(
            visible = False,
            x = X[i*N:(i+1)*N],
            y = y[i*N:(i+1)*N] - model(res.x)[i*N:(i+1)*N],
            mode = 'markers',
            marker =  dict(size= 3),
            name = E.speed_rpm_str + ' residuals'
        ))

    data = trace0 + trace1 + trace2

    data[0]['visible'] = True
    data[nof]['visible'] = True

    steps = []
    for i in range(nof):
        step = dict(
            method = 'restyle',
            args = ['visible', [False] * len(data)],
            label=Expe[i].speed_rpm_str
        )
        step['args'][1][i] = True # Toggle i'th trace to "visible"
        step['args'][1][i+nof] = True # Toggle i'th trace to "visible"
        steps.append(step)

    sliders = [dict(
        active = 0,
        currentvalue = dict(prefix = "Driven angular velocity: "),
        steps = steps,
        pad = {"t": 80}
    )]

    updatemenus = [
        dict(buttons= [
            dict(
                args = ['visible', [True] * (len(data)-nof) + [False] * nof],
                label='See all traces',
                method='restyle'
                ),
            dict(
                args = ['visible', [False] * (len(data)-nof) + [True] * nof],
                label='Residuals',
                method='restyle'
                )],
            direction = 'left',
            pad = {'r': 10, 't': 10},
            showactive = False,
            type = 'buttons',
            x = 0.1,
            xanchor = 'left',
            y = 1,
            yanchor = 'top' 
        )
    ]
    maxF = np.max(model(res.x))
    maxresiduals = np.max(y - model(res.x))
    minresiduals = np.min(y - model(res.x))
    layout = dict(hovermode = 'closest',
                  updatemenus = updatemenus,
                  sliders = sliders, 
                  title = 'Gaussian curve-fitting on the projected data points from the spectrogram analysis on the Frequency-Amplitude plane <br>' + Expe.shaft_desc, 
                  xaxis=dict(title='Freq [Hz]'),
                  yaxis=dict(title='|Y(freq)|', range = [0,maxF])
                  )

    fig = go.Figure(data=data, layout=layout)
    fig.show()
    
    save_plot_button(fig)

In [ ]:
def distribFit4(Expe, yrange=None, log = False):
    fig = make_subplots(rows=5, cols=6, subplot_titles=(Expe.speed_list),
                        horizontal_spacing = 0.01, vertical_spacing=0.05,
                        shared_yaxes=True, x_title='Freq [Hz]', y_title='|Y(freq)|')
    
    nof = Expe.numoffiles
    cut_time = Expe.cut_time
    xdata = []
    ydata = []
    indexdata = []
    fstart = 0.5
    fend = Expe.f*2
    Nstart = int(fstart*cut_time)
    Nend = int(fend*cut_time)
    N = Nend-Nstart
    for i in range(nof):
        xdata.append(Expe[i].fft_freq_1minus2_cut[Nstart:Nend])
        ydata.append(Expe[i].fft_amp_1minus2_cut[Nstart:Nend])
        indexdata.append(np.ones((Nend-Nstart,), dtype=np.int)*(i))
    
    x = np.array(xdata).flatten()
    y = np.array(ydata).flatten()
    k = np.array(indexdata).flatten()
    u = np.column_stack((k,x))
    k = u[:,0].astype(int)
    X = u[:,1]
    mu2v = np.array([Expe[i].speed_rpm_num/60 for i in range(nof)])
    
    # define model function
    def model(x):
        mu1 = x[0]
        sigma1 = x[1]
        sigma2 = x[2]
        C1 = x[k+3]
        C2 = x[k+nof+3]
        mu2 = mu2v[k]
        return np.abs(C1) * np.exp(-(X-mu1)**2/(2*sigma1**2)) + np.abs(C2) * np.exp(-(X-mu2)**2/(2*sigma2**2))

    # define function to optimize
    def func(x):
        return model(x) - y
               
    def funcscal(x):
        return np.sum(np.power((model(x) - y), 2))
    
    
    # define guesses for the simulation
    mu1 = [Expe.f]
    sigma1 = [5]
    sigma2 = [1]
    C1 = [0.05 for i in range(nof)]
    C2 = [Expe[i].speed_rpm_num*0.0002 for i in range(nof)]

    x0 = mu1 + sigma1 + sigma2 + C1 + C2
    a= 0.2
    b= 5
    a1= 0.8
    b1= 1.2
    bmu1 = [(mu1[0]*0.9, mu1[0]*1.1)]
    bsigma1 = [(0, sigma1[0]*b1)]
    bsigma2 = [(0, sigma2[0]*b1)]
    bC1 = [(0, C1[i]*b) for i in range(nof)]
    bC2 = [(0, C2[i]*b) for i in range(nof)]

    bounds = bmu1 + bsigma1 + bsigma2  + bC1 + bC2
    
    # solve the least square optimization
    #res = least_squares(func,x0)
    res = minimize(funcscal,x0, method='Nelder-Mead',tol=1e-6)
    # res = differential_evolution(funcscal, bounds)
    
    # show results
    mu1 = res.x[0]
    sigma1 = res.x[1]
    sigma2 = res.x[2]
    C1 = res.x[3:nof+3]
    C2 = res.x[nof+3:2*nof+3]
    
    k=0
    for i in range(1,6):
        for j in range(1,7):
            E = Expe[k]
            NF = int(E.t1a[-1]*60)
            fig.add_trace(go.Scatter(x = x[k*N:(k+1)*N],
                                     y = y[k*N:(k+1)*N],
                                     mode = 'markers',
                                     marker =  dict(size= 1, color = 'blue'),
                                     name = 'data points',
                                     legendgroup="data",
                                     showlegend = True if k==0 else False
                                    ),
                          row=i, col=j
                         )
            fig.add_trace(go.Scatter(x = x[k*N:(k+1)*N],
                                     y = model(res.x)[i*N:(i+1)*N],
                                     mode = 'lines',
                                     name =  'curve fit',
                                     line = dict( color = 'red', width = 1),
                                     legendgroup="fit",
                                     showlegend = True if k==0 else False
                                    ),
                          row=i, col=j
                         )
            
            
            k+=1

    for a in fig.layout.annotations:
        a.font.size = 12
    if log == False and yrange!=None:
        for i in range(1,31):
            fig.layout[f'yaxis{i}'].range = (0,yrange)
    fig.update_layout(title_text = 'Least square optimization on unfiltered FFT (multiple peaks) -matrix<br>' + Expe.shaft_desc,
                      height=800, width=1000, font_size=10, legend_orientation="h")
    fig.show()
    
    save_plot_button(fig)

#### Healthy shaft

In [ ]:
distribFit4_sliders(HeS)

In [ ]:
distribFit4(HeS)

In [ ]:
distribFit4(CrS)

#### Cracked shaft

In [ ]:
distribFit4_sliders(CrS)

### Multiple peaks, single wedge slope
In the FFT spectrum we can see that the noisy data at low amplitudes tend to increase linearly at higher frequencies. Therefore we can adapt our model and add this further parameter to it. In this case a single slope value is parametrized for all the datasets.

- Model:
$$A(f, rpm, \mu_1,\sigma_1, \sigma_2, C_{1_{rpm}}, C_{2_{rpm}}) =  C_{1_{rpm}} \cdot e^{-\dfrac{(f-\mu_1)^2}{2\sigma_1^2}} + C_{2_{rpm}} \cdot e^{-\dfrac{(f-k\cdot \frac{rpm}{60})^2}{2\sigma_2^2}} + c\cdot f$$ 

where $k$ represent the order to be fitted and $rpm$ the driven angular velocity in rounds per minutes


- Function to minimize:

$$ \sum_{rpm,f}{ \left( FFT(f, rpm) - A(f, rpm, \mu_1,\sigma_1, \sigma_2, C_{1_{rpm}}, C_{2_{rpm}},c) \right)^2 }  $$

The terms to be determined in the optimization are the amplitudes $C_{rpm}$ with $rpm$ ranging from {{HeS.speed_list[0] + ' to ' + HeS.speed_list[-1]}}, the expectation term $\mu$, the standard deviation $\sigma$ and the wedge slope $c$.


In [ ]:
def distribFit41(Expe):
    nof = Expe.numoffiles
    cut_time = Expe.cut_time
    xdata = []
    ydata = []
    indexdata = []
    fstart = 0.5
    fend = Expe.f*1.5
    Nstart = int(fstart*cut_time)
    Nend = int(fend*cut_time)
    N = Nend-Nstart
    for i in range(nof):
        xdata.append(Expe[i].fft_freq_1minus2_cut[Nstart:Nend])
        ydata.append(Expe[i].fft_amp_1minus2_cut[Nstart:Nend])
        indexdata.append(np.ones((Nend-Nstart,), dtype=np.int)*(i))

    x = np.array(xdata).flatten()
    y = np.array(ydata).flatten()
    k = np.array(indexdata).flatten()
    u = np.column_stack((k,x))
    k = u[:,0].astype(int)
    X = u[:,1]
    mu2v = np.array([Expe[i].speed_rpm_num/60 for i in range(nof)])
    # define model function
    def model(x):
        mu1 = x[0]
        sigma1 = x[1]
        sigma2 = x[2]
        c = x[3]
        C1 = x[k+4]
        C2 = x[k+nof+4]
        mu2 = mu2v[k]
        return np.abs(C1) * np.exp(-(X-mu1)**2/(2*sigma1**2)) + np.abs(C2) * np.exp(-(X-mu2)**2/(2*sigma2**2)) + c*X

    # define function to optimize
    def func(x):
        return model(x) - y
               
    def funcscal(x):
        return np.sum(np.power((model(x) - y), 2))
    
    
    # define guesses for the simulation
    mu1 = [Expe.f]
    sigma1 = [5]
    sigma2 = [1]
    c = [0.0001]
    C1 = [0.05 for i in range(nof)]
    C2 = [Expe[i].speed_rpm_num*0.0002 for i in range(nof)]
    x0 = mu1 + sigma1 + sigma2 + c + C1 + C2
    
    # solve the least square optimization
    # res = least_squares(func,x0)
    res = minimize(funcscal,x0, method='Nelder-Mead',tol=1e-6)
    
    # show results
    mu1 = res.x[0]
    sigma1 = res.x[1]
    sigma2 = res.x[2]
    c = res.x[3]
    C1 = res.x[4:nof+4]
    C2 = res.x[nof+4:2*nof+4]
    
    # plot the results in a table
    pd.set_option('display.max_columns', 30)
    table1 = {'Expectation $\mu_1 = f $ [Hz]': mu1,
              'Standard deviation $\sigma1$ [Hz]': sigma1,
              'Standard deviation $\sigma2$ [Hz]': sigma2,
              'Wedge slope $c \cdot 10^3$': c*1000}
    table2 = {'Expectation $\mu_2 $ [Hz]': mu2v,
              'Amplitudes $C_{1_{rpm}}$' : C1,
              'Amplitudes $C_{2_{rpm}}$' : C2}

    df1 = pd.DataFrame(table1, index = ['Value']).round(2)
    df2 = pd.DataFrame(table2,index = Expe.speed_list).T.round(4);
    display(df1,df2)

    #plot the results
    maxF = np.max(model(res.x))
    maxresiduals = np.max(y - model(res.x))
    minresiduals = np.min(y - model(res.x))
    
    trace0 = [] ; trace1 = [] ; trace2 = []

    for i in range(Expe.numoffiles):
        E=Expe[i]
        trace0.append(go.Scatter(
            visible = False,
            x = X[i*N:(i+1)*N],
            y = y[i*N:(i+1)*N],
            mode = 'markers',
            marker =  dict(size= 3),
            name = E.speed_rpm_str + ' data points'
        ))
        trace1.append(go.Scatter(
            visible = False,
            x = X[i*N:(i+1)*N],
            y = model(res.x)[i*N:(i+1)*N],
            mode = 'lines',
            marker= dict(line= dict(width=1)),
            name =  E.speed_rpm_str + ' curve fit'
        ))
        trace2.append(go.Scatter(
            visible = False,
            x = X[i*N:(i+1)*N],
            y = y[i*N:(i+1)*N] - model(res.x)[i*N:(i+1)*N],
            mode = 'markers',
            marker =  dict(size= 3),
            name = E.speed_rpm_str + ' residuals'
        ))

    data = trace0 + trace1 + trace2

    data[0]['visible'] = True
    data[nof]['visible'] = True

    steps = []
    for i in range(nof):
        step = dict(
            method = 'restyle',
            args = ['visible', [False] * len(data)],
            label=Expe[i].speed_rpm_str
        )
        step['args'][1][i] = True # Toggle i'th trace to "visible"
        step['args'][1][i+nof] = True # Toggle i'th trace to "visible"
        steps.append(step)

    sliders = [dict(
        active = 0,
        currentvalue = dict(prefix = "Driven angular velocity: "),
        steps = steps,
        pad = {"t": 80}
    )]

    updatemenus = [
        dict(buttons= [
            dict(
                args = ['visible', [True] * (len(data)-nof) + [False] * nof],
                label='See all traces',
                method='restyle'
                ),
            dict(
                args = ['visible', [False] * (len(data)-nof) + [True] * nof],
                label='Residuals',
                method='restyle'
                ),
            dict(
                args = ['yaxis.range', [minresiduals,maxresiduals]],
                label='range',
                method='relayout'
                )],
            direction = 'left',
            pad = {'r': 10, 't': 10},
            showactive = False,
            type = 'buttons',
            x = 0.1,
            xanchor = 'left',
            y = 1,
            yanchor = 'top' 
        )
    ]
    
    layout = dict(hovermode = 'closest',
                  updatemenus = updatemenus,
                  sliders = sliders, 
                  title = 'Gaussian curve-fitting on the projected data points from the spectrogram analysis on the Frequency-Amplitude plane <br>' + Expe.shaft_desc, 
                  xaxis=dict(title='Freq [Hz]'),
                  yaxis=dict(title='|Y(freq)|', range = [0,maxF])
                  )

    fig = go.Figure(data=data, layout=layout)
    fig.show()
    
    save_plot_button(fig)

#### Healthy shaft

In [ ]:
distribFit41(HeS)

#### Cracked shaft

In [ ]:
distribFit41(CrS)

### Multiple peaks, multiple wedge slopes
In the FFT spectrum we can see that the noisy data at low amplitudes tend to increase linearly at higher frequencies. Therefore we can adapt our model and add this further parameter to it. In this case a slope value is parametrized for each series of  the datasets.

- Model:
$$A(f, rpm, \mu_1,\sigma_1, \sigma_2, C_{1_{rpm}}, C_{2_{rpm}}) =  C_{1_{rpm}} \cdot e^{-\dfrac{(f-\mu_1)^2}{2\sigma_1^2}} + C_{2_{rpm}} \cdot e^{-\dfrac{(f-k\cdot \frac{rpm}{60})^2}{2\sigma_2^2}} + c_{rpm}\cdot f$$ 

where $k$ represent the order to be fitted and $rpm$ the driven angular velocity in rounds per minutes


- Function to minimize:

$$ \sum_{rpm,f}{ \left( FFT(f, rpm) - A(f, rpm, \mu_1,\sigma_1, \sigma_2, C_{1_{rpm}}, C_{2_{rpm}},c_{rpm}) \right)^2 }  $$

The terms to be determined in the optimization are the amplitudes $C_{rpm}$ and the wedge slopes c_{rpm} with $rpm$ ranging from {{HeS.speed_list[0] + ' to ' + HeS.speed_list[-1]}}, the expectation term $\mu$, the standard deviation $\sigma$.


In [ ]:
def distribFit42(Expe):
    nof = Expe.numoffiles
    cut_time = Expe.cut_time
    xdata = []
    ydata = []
    indexdata = []
    fstart = 0.5
    fend = Expe.f*1.5
    Nstart = int(fstart*cut_time)
    Nend = int(fend*cut_time)
    N = Nend-Nstart
    for i in range(nof):
        xdata.append(Expe[i].fft_freq_1minus2_cut[Nstart:Nend])
        ydata.append(Expe[i].fft_amp_1minus2_cut[Nstart:Nend])
        indexdata.append(np.ones((Nend-Nstart,), dtype=np.int)*(i))

    x = np.array(xdata).flatten()
    y = np.array(ydata).flatten()
    k = np.array(indexdata).flatten()
    u = np.column_stack((k,x))
    k = u[:,0].astype(int)
    X = u[:,1]
    mu2v = np.array([Expe[i].speed_rpm_num/60 for i in range(nof)])
    # define model function
    def model(x):
        mu1 = x[0]
        sigma1 = x[1]
        sigma2 = x[2]
        C1 = x[k+3]
        C2 = x[k+nof+3]        
        c = x[k+2*nof+3]
        mu2 = mu2v[k]
        return np.abs(C1) * np.exp(-(X-mu1)**2/(2*sigma1**2)) + np.abs(C2) * np.exp(-(X-mu2)**2/(2*sigma2**2)) + c*X

    # define function to optimize
    def func(x):
        return model(x) - y
               
    def funcscal(x):
        return np.sum(np.power((model(x) - y), 2))
    
    
    # define guesses for the simulation
    mu1 = [Expe.f]
    sigma1 = [5]
    sigma2 = [1]
    C1 = [0.05 for i in range(nof)]
    C2 = [Expe[i].speed_rpm_num*0.0002 for i in range(nof)]
    c = [0.0001 for i in range(nof)]
    x0 = mu1 + sigma1 + sigma2 + C1 + C2 + c
    
    # solve the least square optimization
    # res = least_squares(func,x0)
    res = minimize(funcscal,x0, method='Nelder-Mead',tol=1e-6)
    
    # show results
    mu1 = res.x[0]
    sigma1 = res.x[1]
    sigma2 = res.x[2]
    C1 = res.x[3:nof+3]
    C2 = res.x[nof+3:2*nof+3]
    c = res.x[2*nof+3:3*nof+3]

    # plot the results in a table
    pd.set_option('display.max_columns', 30)
    table1 = {'Expectation $\mu_1 = f $ [Hz]': mu1,
              'Standard deviation $\sigma1$ [Hz]': sigma1,
              'Standard deviation $\sigma2$ [Hz]': sigma2}
    table2 = {'Expectation $\mu_2$ [Hz]': mu2v,
              'Amplitudes $C_{1_{rpm}}$' : C1,
              'Amplitudes $C_{2_{rpm}}$' : C2,
              'Wedge slope $c\cdot 10^3$': c*1000}

    df1 = pd.DataFrame(table1, index = ['Value']).round(2)
    df2 = pd.DataFrame(table2,index = Expe.speed_list).T.round(4);
    display(df1,df2)

    #plot the results
    maxF = np.max(model(res.x))
    maxresiduals = np.max(y - model(res.x))
    minresiduals = np.min(y - model(res.x))
    
    trace0 = [] ; trace1 = [] ; trace2 = []

    for i in range(Expe.numoffiles):
        E=Expe[i]
        trace0.append(go.Scatter(
            visible = False,
            x = X[i*N:(i+1)*N],
            y = y[i*N:(i+1)*N],
            mode = 'markers',
            marker =  dict(size= 5),
            name = E.speed_rpm_str + ' data points'
        ))
        trace1.append(go.Scatter(
            visible = False,
            x = X[i*N:(i+1)*N],
            y = model(res.x)[i*N:(i+1)*N],
            mode = 'lines',
            marker= dict(line= dict(width=1)),
            name =  E.speed_rpm_str + ' curve fit'
        ))
        trace2.append(go.Scatter(
            visible = False,
            x = X[i*N:(i+1)*N],
            y = y[i*N:(i+1)*N] - model(res.x)[i*N:(i+1)*N],
            mode = 'markers',
            marker =  dict(size= 5),
            name = E.speed_rpm_str + ' residuals'
        ))

    data = trace0 + trace1 + trace2

    data[0]['visible'] = True
    data[nof]['visible'] = True

    steps = []
    for i in range(nof):
        step = dict(
            method = 'restyle',
            args = ['visible', [False] * len(data)],
            label=Expe[i].speed_rpm_str
        )
        step['args'][1][i] = True # Toggle i'th trace to "visible"
        step['args'][1][i+nof] = True # Toggle i'th trace to "visible"
        steps.append(step)

    sliders = [dict(
        active = 0,
        currentvalue = dict(prefix = "Driven angular velocity: "),
        steps = steps,
        pad = {"t": 80}
    )]

    updatemenus = [
        dict(buttons= [
            dict(
                args = ['visible', [True] * (len(data)-nof) + [False] * nof],
                label='See all traces',
                method='restyle'
                ),
            dict(
                args = ['visible', [False] * (len(data)-nof) + [True] * nof],
                label='Residuals',
                method='restyle'
                ),
            dict(
                args = ['yaxis.range', [minresiduals,maxresiduals]],
                label='range',
                method='relayout'
                )],
            direction = 'left',
            pad = {'r': 10, 't': 10},
            showactive = False,
            type = 'buttons',
            x = 0.1,
            xanchor = 'left',
            y = 1,
            yanchor = 'top' 
        )
    ]
    
    layout = dict(hovermode = 'closest',
                  updatemenus = updatemenus,
                  sliders = sliders, 
                  title = 'Gaussian curve-fitting on the projected data points from the spectrogram analysis on the Frequency-Amplitude plane <br>' + Expe.shaft_desc, 
                  xaxis=dict(title='Freq [Hz]'),
                  yaxis=dict(title='|Y(freq)|', range = [0,maxF])
                  )

    fig = go.Figure(data=data, layout=layout)
    fig.show()
    
    save_plot_button(fig)

#### Healthy shaft

In [ ]:
distribFit42(HeS)

#### Cracked shaft

In [ ]:
distribFit42(CrS)

# References

[1] 
<cite data-cite="groover2005">Groover et al., Removal of order domain content in rotating equipment signals by double resampling, Mechanical Systems and Signal Processing, 19 (2005) 483–500 </cite>